# III. Creation de Features

Une fois que vous avez identifié un ensemble de features présentant un certain potentiel, il est temps de commencer à les développer. Dans ce cours, vous apprendrez un certain nombre de transformations courantes que vous pouvez effectuer avec Pandas.

Nous utiliserons quatre ensembles de données ayant une gamme de types de features : *US Traffic Accidents*, *1985 Automobiles*, *Concrete Formulations* et *Customer Lifetime Value*.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

accidents = pd.read_csv("../Data/data2/FE-Course-Data/accidents.csv")
autos = pd.read_csv("../Data/data2/FE-Course-Data/autos.csv")
concrete = pd.read_csv("../Data/data2/FE-Course-Data/concrete.csv")
customer = pd.read_csv("../Data/data2/FE-Course-Data/customer.csv")

<blockquote style="margin-right:auto; margin-left:auto; background-color: #ebf9ff; padding: 1em; margin:24px;">
<strong>Tips on Discovering New Features</strong>
<ul>
<li>Understand the features. Refer to your dataset's <em>data documentation</em>, if available.
<li>Research the problem domain to acquire <strong>domain knowledge</strong>. If your problem is predicting house prices, do some research on real-estate for instance. Wikipedia can be a good starting point, but books and <a href="https://scholar.google.com/">journal articles</a> will often have the best information.
<li>Study previous work.
<li>Use data visualization. Visualization can reveal pathologies in the distribution of a feature or complicated relationships that could be simplified. Be sure to visualize your dataset as you work through the feature engineering process.
<ul>
</blockquote>

### Transformations mathématiques #

Les relations entre les features numériques sont souvent exprimées par des formules mathématiques, que vous rencontrerez fréquemment dans le cadre de votre recherche de domaine. Dans Pandas, vous pouvez appliquer des opérations arithmétiques aux colonnes comme s'il s'agissait de nombres ordinaires.

Dans le dataset *Automobile* se trouvent des features décrivant le moteur d'une voiture. La recherche fournit une variété de formules pour créer de nouvelles features potentiellement utiles. Le "rapport de course", par exemple, est une mesure de l'efficacité d'un moteur par rapport à sa performance :

In [ ]:
autos["stroke_ratio"] = autos.stroke / autos.bore

autos[["stroke", "bore", "stroke_ratio"]].head()

Plus une combinaison est compliquée, plus il sera difficile pour un modèle d'apprendre, comme cette formule pour la « cylindrée » d'un moteur, une mesure de sa puissance :

In [ ]:
autos["displacement"] = (
    np.pi * ((0.5 * autos.bore) ** 2) * autos.stroke * autos.num_of_cylinders
)

La visualisation des données peut suggérer des transformations, souvent un "reshaping" d'une feature à l'aide de puissances ou de logarithmes. La distribution de `WindSpeed` dans *US Accidents* est très asymétrique, par exemple. Dans ce cas, le logarithme est efficace pour le normaliser :

In [ ]:
# If the feature has 0.0 values, use np.log1p (log(1+x)) instead of np.log
accidents["LogWindSpeed"] = accidents.WindSpeed.apply(np.log1p)

# Plot a comparison
fig, axs = plt.subplots(1, 2, figsize=(8, 4))
sns.kdeplot(accidents.WindSpeed, shade=True, ax=axs[0])
sns.kdeplot(accidents.LogWindSpeed, shade=True, ax=axs[1]);

### Counts #

Les caractéristiques décrivant la présence ou l'absence de quelque chose viennent souvent en ensembles, par exemple l'ensemble des facteurs de risque d'une maladie. Vous pouvez agréger ces caractéristiques en créant un **compte**.

Ces caractéristiques seront *binaires* (`1` pour Présent, `0` pour Absent) ou *booléen* (`True` ou `False`). En Python, les booléens peuvent être additionnés comme s'il s'agissait d'entiers.

Dans *Traffic Accidents*, plusieurs éléments indiquent si un objet de la chaussée était à proximité de l'accident. Cela créera un décompte du nombre total d'entités routières à proximité à l'aide de la méthode `sum` :

In [ ]:
roadway_features = ["Amenity", "Bump", "Crossing", "GiveWay",
    "Junction", "NoExit", "Railway", "Roundabout", "Station", "Stop",
    "TrafficCalming", "TrafficSignal"]
accidents["RoadwayFeatures"] = accidents[roadway_features].sum(axis=1)

accidents[roadway_features + ["RoadwayFeatures"]].head(10)

Vous pouvez également utiliser les méthodes intégrées d'un cadre de données pour *créer* des valeurs booléennes. Dans l'ensemble de données *Concrete* se trouvent les quantités de composants dans une formulation concrète. De nombreuses formulations manquent d'un ou plusieurs composants (c'est-à-dire que le composant a une valeur de 0). Cela comptera le nombre de composants dans une formulation avec la méthode intégrée supérieure à `gt` :

In [ ]:
components = [ "Cement", "BlastFurnaceSlag", "FlyAsh", "Water",
               "Superplasticizer", "CoarseAggregate", "FineAggregate"]
concrete["Components"] = concrete[components].gt(0).sum(axis=1)

concrete[components + ["Components"]].head(10)

### Construction et décomposition de features #

Souvent, vous aurez des chaînes complexes qui peuvent utilement être divisées en morceaux plus simples. Quelques exemples courants :
- ID numbers: `'123-45-6789'`
- Phone numbers: `'(999) 555-0123'`
- Street addresses: `'8241 Kaggle Ln., Goose City, NV'`
- Internet addresses: `'http://www.kaggle.com`
- Product codes: `'0 36000 29145 2'`
- Dates and times: `'Mon Sep 30 07:06:05 2013'`

Des features comme celles-ci auront souvent une sorte de structure que vous pourrez utiliser. Les numéros de téléphone américains, par exemple, ont un indicatif régional (la partie `'(999)'`) qui vous indique l'emplacement de l'appelant. Comme toujours, certaines recherches peuvent être payantes ici.

L'accesseur `str` vous permet d'appliquer des méthodes directement sur les colonnes telle que `split`. L'ensemble de données *Customer Lifetime Value* contient des features décrivant les clients d'une compagnie d'assurance. À partir de la feature `Policy`, nous pourrions séparer le « Type » du « Niveau » de couverture :

In [ ]:
customer[["Type", "Level"]] = (  # Create two new features
    customer["Policy"]           # from the Policy feature
    .str                         # through the string accessor
    .split(" ", expand=True)     # by splitting on " "
                                 # and expanding the result into separate columns
)

customer[["Policy", "Type", "Level"]].head(10)

Vous pouvez également regrouper des features simples dans une feature composée si vous avez des raisons de croire qu'il y a eu une interaction dans la combinaison :

In [ ]:
autos["make_and_style"] = autos["make"] + "_" + autos["body_style"]
autos[["make", "body_style", "make_and_style"]].head()

### Group Transforms #

Enfin nous avons les **transformations de groupe**, qui agrège les informations sur plusieurs lignes regroupées par catégorie. Avec une transformation de groupe, vous pouvez créer des features telles que : "le revenu moyen de l'état de résidence d'une personne" ou "la proportion de films sortis un jour de semaine, par genre." Si vous aviez découvert une interaction de catégorie, une transformation de groupe sur cette catégorie pourrait être quelque chose de bon à étudier.

À l'aide d'une fonction d'agrégation, une transformation de groupe combine deux features : une feature catégorielle qui fournit le regroupement et une autre feature dont vous souhaitez agréger les valeurs. Pour un « revenu moyen par état », vous choisiriez `State` pour la feature de regroupement, `mean` pour la fonction d'agrégation et `Income` pour la feature agrégée. Pour calculer cela dans Pandas, nous utilisons les méthodes « groupby » et « transform » :

In [ ]:
customer["AverageIncome"] = (
    customer.groupby("State")  # for each state
    ["Income"]                 # select the income
    .transform("mean")         # and compute its mean
)

customer[["State", "Income", "AverageIncome"]].head(10)

Voici comment vous pouvez calculer la fréquence à laquelle chaque état se produit dans l'ensemble de données :

In [ ]:
customer["StateFreq"] = (
    customer.groupby("State")
    ["State"]
    .transform("count")
    / customer.State.count()
)

customer[["State", "StateFreq"]].head(10)

Vous pouvez utiliser une transformation comme celle-ci pour créer un "codage de fréquence" pour une caractéristique catégorielle.

Si vous utilisez des fractionnements d'entraînement et de validation, pour préserver leur indépendance, il est préférable de créer une feature groupée en utilisant uniquement l'ensemble d'entraînement, puis de la joindre à l'ensemble de validation. Nous pouvons utiliser la méthode « merge » de l'ensemble de validation après avoir créé un ensemble unique de valeurs avec « drop_duplicates » sur l'ensemble d'apprentissage :

In [ ]:
# Create splits
df_train = customer.sample(frac=0.5)
df_valid = customer.drop(df_train.index)

# Create the average claim amount by coverage type, on the training set
df_train["AverageClaim"] = df_train.groupby("Coverage")["ClaimAmount"].transform("mean")

# Merge the values into the validation set
df_valid = df_valid.merge(
    df_train[["Coverage", "AverageClaim"]].drop_duplicates(),
    on="Coverage",
    how="left",
)

df_valid[["Coverage", "AverageClaim"]].head(10)

<blockquote style="margin-right:auto; margin-left:auto; background-color: #ebf9ff; padding: 1em; margin:24px;">
<strong>Tips on Creating Features</strong><br>
It's good to keep in mind your model's own strengths and weaknesses when creating features. Here are some guidelines:
<ul>
<li> Linear models learn sums and differences naturally, but can't learn anything more complex.
<li> Ratios seem to be difficult for most models to learn. Ratio combinations often lead to some easy performance gains.
<li> Linear models and neural nets generally do better with normalized features. Neural nets especially need features scaled to values not too far from 0. Tree-based models (like random forests and XGBoost) can sometimes benefit from normalization, but usually much less so.
<li> Tree models can learn to approximate almost any combination of features, but when a combination is especially important they can still benefit from having it explicitly created, especially when data is limited.
<li> Counts are especially helpful for tree models, since these models don't have a natural way of aggregating information across many features at once.
</ul>
</blockquote>

## 2. Exercice

**Vous serez notés sur cet exercice.**

**C'est un travail individuel.**

**Le notebook contenant votre code et le résultat de votre exécution est à envoyer à mghesmoune@gmail.com avec comme objet de mail [`creation_features_votre-nom`]**

Dans cet exercice, vous commencerez à développer les features que vous avez identifiées dans l'exercice précédent comme ayant le plus de potentiel. Au cours de cet exercice, vous pouvez prendre un moment pour examiner à nouveau la documentation des données et vous demander si les fonctionnalités que nous créons ont du sens d'un point de vue réel et s'il existe des combinaisons utiles qui vous semblent intéressantes.

In [ ]:
!pip install xgboost

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor


def score_dataset(X, y, model=XGBRegressor()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_log_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score


# Prepare data
df = pd.read_csv("../Data/data2/FE-Course-Data/ames.csv")
X = df.copy()
y = X.pop("SalePrice")

-------------------------------------------------------------------------------
Commençons par quelques combinaisons mathématiques. Nous nous concentrerons sur les features décrivant les zones -- ayant les mêmes unités permet de les combiner facilement de manière judicieuse. Puisque nous utilisons XGBoost (a tree-based model), nous allons nous concentrer sur les ratios et les sommes.

### 1) Create Mathematical Transforms

Créer les features suivantes :

- `LivLotRatio`: the ratio of `GrLivArea` to `LotArea`
- `Spaciousness`: the sum of `FirstFlrSF` and `SecondFlrSF` divided by `TotRmsAbvGrd`
- `TotalOutsideSF`: the sum of `WoodDeckSF`, `OpenPorchSF`, `EnclosedPorch`, `Threeseasonporch`, and `ScreenPorch`

In [ ]:
# YOUR CODE HERE
X_1 = pd.DataFrame()  # dataframe to hold new features

X_1["LivLotRatio"] = ____
X_1["Spaciousness"] = ____
X_1["TotalOutsideSF"] = ____


-------------------------------------------------------------------------------
Si vous avez découvert un effet d'interaction entre une feature numérique et une feature catégorielle, vous souhaiterez peut-être le modéliser explicitement à l'aide d'un encodage one-hot, comme ceci :

```
# One-hot encode Categorical feature, adding a column prefix "Cat"
X_new = pd.get_dummies(df.Categorical, prefix="Cat")

# Multiply row-by-row
X_new = X_new.mul(df.Continuous, axis=0)

# Join the new features to the feature set
X = X.join(X_new)
```

### 2) Interaction with a Categorical

Nous avons découvert une interaction entre `BldgType` et `GrLivArea` dans l'exercice 2. Créez maintenant leurs features d'interaction. 

In [ ]:
# YOUR CODE HERE
# One-hot encode BldgType. Use `prefix="Bldg"` in `get_dummies`
X_2 = ____ 
# Multiply
X_2 = ____

### 3) Count Feature

Essayons de créer une feature qui décrit le nombre de types d'espaces extérieurs dont dispose une habitation. Créez une feature « PorchTypes » qui compte combien des éléments suivants sont supérieurs à 0,0 :

```
WoodDeckSF
OpenPorchSF
EnclosedPorch
Threeseasonporch
ScreenPorch
```

In [ ]:
X_3 = pd.DataFrame()

# YOUR CODE HERE
X_3["PorchTypes"] = ____


### 4) Break Down a Categorical Feature

`MSSubClass` décrit le type de logement :

In [ ]:
df.MSSubClass.unique()

Vous pouvez voir qu'il existe une catégorisation plus générale décrite (en gros) par le premier mot de chaque catégorie. Créez une feature contenant uniquement ces premiers mots en divisant `MSSubClass` au premier trait de soulignement `_`. (Indice : dans la méthode `split`, utilisez un argument `n=1`.)

In [ ]:
X_4 = pd.DataFrame()

# YOUR CODE HERE
____

### 5) Use a Grouped Transform

La valeur d'une maison dépend souvent de la façon dont elle se compare aux maisons typiques de son quartier. Créez une fonction `MedNhbdArea` qui décrit la *médiane* de `GrLivArea` regroupée sur `Neighborhood`.

In [ ]:
X_5 = pd.DataFrame()

# YOUR CODE HERE
X_5["MedNhbdArea"] = ____


Vous avez maintenant créé votre premier ensemble de nouvelles features ! Vous pouvez exécuter la cellule ci-dessous pour évaluer le modèle avec toutes vos nouvelles features ajoutées :

In [ ]:
X_new = X.join([X_1, X_2, X_3, X_4, X_5])
score_dataset(X_new, y)